# Package Imports

In [ ]:
import pandas as pd

# Loading Dataset

In [ ]:
diabetes = pd.read_csv('/content/diabetic_data.csv')

In [ ]:
diabetes.info()

In [ ]:
diabetes.head()

# Cleaning Scripts

In [ ]:
# Replacing the 'ch' values in the 'change' column to 'change'
Data['change'] = Data['change'].str.replace('Ch', 'change')

In [ ]:
# Defining a function to group the values in a column by a specified range
def group_by_range(df, col_name, n):
    df_sorted = df.sort_values(col_name)
    min_val = df_sorted[col_name].min()
    max_val = df_sorted[col_name].max()
    num_groups = int((max_val - min_val) / n) + 1
    group_labels = [f"{min_val + i*n}-{min_val + (i+1)*n-1}" for i in range(num_groups)]
    df_sorted[col_name] = pandas.cut(df_sorted[col_name], bins=num_groups, labels=group_labels, include_lowest=True)
    return df_sorted


In [ ]:
# Calling the function to group the values in the specified columns by a specified range
Data = group_by_range(Data, 'time_in_hospital', 3)
Data = group_by_range(Data, 'num_lab_procedures', 5)
Data = group_by_range(Data, 'num_medications', 5)
Data = group_by_range(Data, 'number_outpatient', 5)
Data = group_by_range(Data, 'number_emergency', 5)
Data = group_by_range(Data, 'number_inpatient', 3)
Data = group_by_range(Data, 'number_diagnoses', 3)

In [ ]:
# Remapping the admission_type_id, discharge_disposition_id, and admission_source_id columns with the data dictionaries in the IDs_mapping.xlsx file
file_path = "data\IDs_mapping.xlsx"
wb = load_workbook(filename = file_path)
DataDictionaries = {}
for sheet_name in wb.sheetnames:
    df = pandas.read_excel(file_path, sheet_name=sheet_name)
    dict_name = sheet_name.replace(" ", "_").lower()
    data_dict = dict(zip(df.iloc[:, 0], df.iloc[:, 1]))
    DataDictionaries[dict_name] = data_dict
for dict_name, data_dict in DataDictionaries.items():
    matching_cols = [col for col in Data.columns if col == dict_name]
    for col in matching_cols:
        Data[col] = Data[col].map(data_dict)

In [ ]:
# Replacing the 'E' values in the 'diag_1', 'diag_2', and 'diag_3' columns to '10' to make conversion to float possible
Data['diag_1'] = Data['diag_1'].astype(str).str.replace('E', '10')
Data['diag_2'] = Data['diag_2'].astype(str).str.replace('E', '10')
Data['diag_3'] = Data['diag_3'].astype(str).str.replace('E', '10')
# Replacing the 'E' values in the 'diag_1', 'diag_2', and 'diag_3' columns to '10' to make conversion to float possible
Data['diag_1'] = Data['diag_1'].astype(str).str.replace('V', '20')
Data['diag_2'] = Data['diag_2'].astype(str).str.replace('V', '20')
Data['diag_3'] = Data['diag_3'].astype(str).str.replace('V', '20')
# Converting the 'diag_1', 'diag_2', and 'diag_3' columns to float
Data['diag_1'] = Data['diag_1'].apply(lambda x: numpy.ceil(float(x)) if x != '?' else x)
Data['diag_2'] = Data['diag_2'].apply(lambda x: numpy.ceil(float(x)) if x != '?' else x)
Data['diag_3'] = Data['diag_3'].apply(lambda x: numpy.ceil(float(x)) if x != '?' else x)

In [ ]:
# Defining a dictionary that maps ICD code ranges to their associated descriptions
icd_code_ranges = {
    range(1, 140): 'INFECTIOUS AND PARASITIC DISEASES',
    range(140, 240): 'NEOPLASMS',
    range(240, 280): 'ENDOCRINE, NUTRITIONAL AND METABOLIC DISEASES, AND IMMUNITY DISORDERS',
    range(280, 290): 'DISEASES OF THE BLOOD AND BLOOD-FORMING ORGANS',
    range(290, 320): 'MENTAL, BEHAVIORAL AND NEURODEVELOPMENTAL DISORDERS',
    range(320, 390): 'DISEASES OF THE NERVOUS SYSTEM AND SENSE ORGANS',
    range(390, 460): 'DISEASES OF THE CIRCULATORY SYSTEM',
    range(460, 520): 'DISEASES OF THE RESPIRATORY SYSTEM',
    range(520, 580): 'DISEASES OF THE DIGESTIVE SYSTEM',
    range(580, 630): 'DISEASES OF THE GENITOURINARY SYSTEM',
    range(630, 680): 'COMPLICATIONS OF PREGNANCY, CHILDBIRTH, AND THE PUERPERIUM',
    range(680, 710): 'DISEASES OF THE SKIN AND SUBCUTANEOUS TISSUE',
    range(710, 740): 'DISEASES OF THE MUSCULOSKELETAL SYSTEM AND CONNECTIVE TISSUE',
    range(740, 760): 'CONGENITAL ANOMALIES',
    range(760, 780): 'CERTAIN CONDITIONS ORIGINATING IN THE PERINATAL PERIOD',
    range(780, 800): 'SYMPTOMS, SIGNS, AND ILL-DEFINED CONDITIONS',
    range(800, 1000): 'INJURY AND POISONING',
    range(10000, 10999): 'SUPPLEMENTARYCLASSIFICATION OF EXTERNAL CAUSES OF INJURY AND POISONING',
    range(2000, 2099): 'SUPPLEMENTARY CLASSIFICATION OF FACTORS INFLUENCING HEALTH STATUS AND CONTACT WITH HEALTH SERVICES',
}
# Defining a function that converts the values within the 'diag_1', 'diag_2', and 'diag_3' columns to their ICD-9 code descriptions
def icd_code_to_description(icd_code):
    if icd_code == '?':  # if value is '?', return it unchanged
        return icd_code
    for code_range, description in icd_code_ranges.items():
        if int(icd_code) in code_range:
            return description
    return 'UNKNOWN'

In [ ]:
# Applying the icd_code_to_description function to the 'diag_1', 'diag_2', and 'diag_3' columns
Data['diag_1'] = Data['diag_1'].apply(icd_code_to_description)
Data['diag_2'] = Data['diag_2'].apply(icd_code_to_description)
Data['diag_3'] = Data['diag_3'].apply(icd_code_to_description)


In [ ]:
# Replacing '?' and empty cells with NaN 
Data.replace('?', pandas.np.nan, inplace=True)
Data.replace('', pandas.np.nan, inplace=True)
# Counting the number of NaN values remaining in each column 
print(Data.isna().sum())